<a href="https://colab.research.google.com/github/luqmanhaqim/Haqim-repo/blob/main/Billing_Generator_Verify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Files required to be uploaded**


1.   Hana summary report
2.   Customer_Data
3.   contact search result




In [3]:
import pandas as pd
import numpy as np
import openpyxl
import json 
from datetime import datetime, timedelta, date
import pytz
from pytz import timezone

#merge data to generate billing 
#required files as stated in bracket 
def merge_data(extracted_data_csv, customer_data_csv, df_csr_csv):
    extracted_data = pd.read_excel(extracted_data_csv)
    customer_data = pd.read_csv(customer_data_csv)
    df_csr = pd.read_csv(df_csr_csv)

    merged_df = pd.merge(extracted_data, customer_data, on="Policy_Number", how="left")

    #extracts time
    def extract_time(timestamp):
        if pd.isna(timestamp):
            return None
        timestamp = str(timestamp)
        timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        timestamp = datetime.strftime(timestamp, "%d/%m/%y %H:%M")
        return timestamp.split(" ")[1]

    #extracts date
    def extract_date(timestamp):
        if pd.isna(timestamp):
            return None
        timestamp = str(timestamp)
        timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        timestamp = datetime.strftime(timestamp, "%d/%m/%y %H:%M")
        return timestamp.split(" ")[0]

    
    #Apply defined function to merged_df to extract time and date 
    merged_df['Call_Time'] = merged_df['Trigger_Timestamp'].apply(extract_time)
    merged_df['Trigger_Timestamp'] = merged_df['Trigger_Timestamp'].apply(extract_date)
    
    merged_df = merged_df[['ID', 'Policy_Number', 'Policyholder_Phone_Number_y', 'Entity',
                           'Call_Answered', 'Stages_Completed', 'Verification',
                           'Trigger_Timestamp', "Call_Time", 'Policy_Received', 'Survey_Rating']]

    merged_df.rename(columns={'Policyholder_Phone_Number_y': 'Customer phone number',
                               'Trigger_Timestamp': 'Date'}, inplace=True)
    

    merged_df.drop(merged_df.columns[[0, 6, 9, 10]], axis=1, inplace=True)

    df=merged_df[(merged_df['Date'] <= "2023-02-17")]
    df['Customer phone number'] = "'" + df['Customer phone number'].astype(str)

    df_csr['Call_Time'] = df_csr['Initiation timestamp'].apply(extract_time)
    df_csr['Date'] = df_csr['Initiation timestamp'].apply(extract_date)
    
    df_csr.drop(df_csr.columns[[0, 1, 2, 3, 4, 5, 7]], axis=1, inplace=True)

    merged_df = df.merge(df_csr, on=["Customer phone number", "Date"], how="left")

    policy_numbers = merged_df["Policy_Number"].unique().tolist()

    #Counts for attemp No. to be reviewed 
    for policy_number in policy_numbers:
      policy_rows = merged_df[merged_df["Policy_Number"] == policy_number]
      for i, row in policy_rows.iterrows():
          merged_df.at[i, "Attempt No."] = (policy_rows["Policy_Number"] == policy_number).sum()

    #sort by Y 
    merged_df=merged_df.sort_values(by="Call_Time_y")
    #drop irrelevant columns 
    merged_df=merged_df.drop(["Call_Time_x"],axis=1)
    #change column names 
    merged_df.rename(columns={'Call_Time_y': 'Call Time',
                               'Policy_Number': 'Policy Number'}, inplace=True)
    

    #split answered and unanswered calls 
    #Answered calls FTA
    answered_calls_FTA = merged_df[(merged_df["Entity"] == "FTA") &
                               (~((merged_df["Call_Answered"] == False) | 
                                 ((merged_df["Call_Answered"] == True) & 
                                  (merged_df["Stages_Completed"] == "T.1")))
                               )]

    #drop irrelevent columns answered calls 
    answered_calls_FTA=answered_calls_FTA.drop(["Call_Answered","Stages_Completed","Entity"],axis=1)

    #unanswered calls FTA
    unanswered_calls_FTA= merged_df[(merged_df["Entity"]=="FTA")&((merged_df["Call_Answered"]==False) | ((merged_df["Call_Answered"]==True) & (merged_df["Stages_Completed"]=="T.1")))]
    unanswered_calls_FTA=unanswered_calls_FTA.drop(["Call_Answered","Stages_Completed","Contact duration","Entity"],axis=1)

    #unanswered calls LIA 
    # unanswered_calls_LIA= merged_df[(merged_df["Entity"]=="LIA")&((merged_df["Call_Answered"]==False) | ((merged_df["Call_Answered"]==True) & (merged_df["Stages_Completed"]=="T.1")))]
    # unanswered_calls_LIA=unanswered_calls_FTA.drop(["Call_Answered","Stages_Completed","Call Time","Contact duration"],axis=1)


    print(unanswered_calls_FTA)
    print(len(unanswered_calls_FTA))

    # print(unanswered_calls_LIA)
    # print(len(unanswered_calls_LIA))

    answered_calls_FTA.to_csv("Hana_Billing.csv", index=False)

    return unanswered_calls_FTA,answered_calls_FTA

#generates excel 
def toExcel(unanswered_calls_FTA,answered_calls_FTA,strf):
    with pd.ExcelWriter(f'Hana_Billing {strf}.xlsx') as writer:
    
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
      answered_calls_FTA.to_excel(writer, sheet_name=f'Answered Calls', index=False)
      unanswered_calls_FTA.to_excel(writer, sheet_name=f'Unanswered Calls', index=False)
      
#gets current time
def currentTime():
  now = datetime.now(timezone("Singapore"))
  print(now)
  result1= now
  strf= datetime.strftime(result1,'%Y-%m-%d %I %p')
  print(strf)
  return strf

#cleans contact search extracted data 
def cleaningCSR(x):

  def extract_tim(timestamp):
    if pd.isna(timestamp):
        return None
    timestamp = str(timestamp)
    timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    timestamp = datetime.strftime(timestamp, "%H")
    return timestamp

  def extract_dat(timestamp):
      if pd.isna(timestamp):
          return None
      timestamp = str(timestamp)
      timestamp = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
      timestamp = datetime.strftime(timestamp, "%d/%m/%y %H:%M")
      return timestamp.split(" ")[0]
    
  x = pd.read_csv(f'{x}')
  x=x.sort_values(by=["Disconnect timestamp","Contact duration"],ascending=False)
  x['Call_Time'] = x['Initiation timestamp'].apply(extract_tim)
  x['Date'] = x['Initiation timestamp'].apply(extract_dat)
  x=x.sort_values(by=["Customer phone number","Contact duration"],ascending=False)
  x=x.drop_duplicates(subset=["Customer phone number","Date","Call_Time"],keep="first")
  x=x.drop(columns=["Date","Call_Time"])
  x=x.to_csv("ContactSearchResults.csv",index=False)
  
  return x

    
def main():
  unanswered_calls_FTA,answered_calls_FTA= merge_data("/content/Hana_Call_Summary (compiled).xlsx", "/content/Customer_Data (3).csv", "/content/ContactSearchResults.csv")
  strf=currentTime()
  toExcel(unanswered_calls_FTA,answered_calls_FTA,strf)

#input contact search result in " "
cleaningCSR("/content/ContactSearchResults (27).csv")

# **Run to generate Billing**

In [4]:
main ()

     Policy Number Customer phone number      Date Call Time  Attempt No.
0    LR00000269994         '+60197238838  14/02/23     09:00          3.0
900  LR00000271230         '+60197778826  16/02/23     09:01          3.0
3    LR00000270026         '+60174660220  14/02/23     09:02          2.0
908  LR00000272261         '+60182935100  16/02/23     09:02          4.0
493  LR00000270322         '+60196295350  15/02/23     09:02          3.0
..             ...                   ...       ...       ...          ...
491  TR00003181365         '+60173655097  14/02/23     17:27          3.0
489  TR00003181401         '+60194350992  14/02/23     17:28          3.0
490  TR00003181462         '+60189129305  14/02/23     17:29          4.0
393  TR00003178553         '+60193131506  14/02/23       NaN          3.0
878  TR00003180462         '+60175210721  15/02/23       NaN          3.0

[1117 rows x 5 columns]
1117
2023-02-22 09:28:25.810456+08:00
2023-02-22 09 AM
